# __Web Scraping desde la Practica__

## Importar librerias

In [57]:
import requests
from bs4 import BeautifulSoup
import csv

## Hacer peticiones HTTP

### Conceptos de interes

#### _HTTP:_

- Hypertext Transfer Protocol.
- Es el protocolo de comunicación que permite las transferencias de información a través de archivos en la World Wide Web.

#### _GET:_

- Recupera datos del servidor.
- Se usa para leer o consultar información. No modifica nada.

#### _POST:_

- Envia datos al servidor.
- Se usa para crear nuevos recursos (por ejemplo, enviar un formulario)

#### _PUT:_

- Actualiza un recurso existente.
- Reemplaza por completo el recurso con la nueva información enviada.

#### _DELETE:_

- Elimina un recurso del servidor.
- Se usa para borrar datos específicos.

### Implementacion de peticiones

In [58]:
# guardamos la url de la pagina a una variable
url = 'https://books.toscrape.com/'

# hacemos la peticion con request
response = requests.get(url)

In [59]:
# verificar el codigo de estado
print(response)

<Response [200]>


In [60]:
# acceder a la peticion en formato de texto
print(response.text)

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

> Para visualizar el formato de texto de la pagina dese el navegador presionamos ctrl + u

Ahora vamos a mostrar los 15 primeros caracteres en formato de texto de la pagina solo si el codigo de estado es exitoso.

In [61]:
if response.status_code == 200:
    print('Status Code 200 Exitoso')
    print(response.text[0:15])
else:
    print('Status Code: ', response.status_code)

Status Code 200 Exitoso
<!DOCTYPE html>


Probemos el codigo con una pagina que no existe

In [62]:
url = 'https://books.toscrape.com/libro_ficcion/'

# hacemos la peticion con request
response2 = requests.get(url)

# verificar el codigo de estado
print(response2)

<Response [404]>


In [63]:
# acceder a la peticion en formato de texto
print(response2.text)

<html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx/1.21.6</center>
</body>
</html>



In [64]:
if response2.status_code == 200:
    print('Status Code 200 Exitoso')
    print(response2.text[0:15])
else:
    print('Status Code: ', response2.status_code)

Status Code:  404


## Explorar la estructura HTML con BeautifulSoup

In [65]:
# definimos la url de la pagina
url = 'https://books.toscrape.com/'

# realizar la peticion GET
response = requests.get(url)

# parseamos la peticion para darle un formato HTML
soup = BeautifulSoup(response.text, 'html.parser')

# mostrar resultado
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

> Pareciera que devuelve el mismo formato que la peticion anterior. Sin embargo, esta estructura permite acceder a los elementos a traves de las etiquetas.

In [66]:
# extraer el head
head = soup.find('title')
print(head)

<title>
    All products | Books to Scrape - Sandbox
</title>


In [67]:
# extraer solo el texto de head
head = soup.find('title')
print(head.get_text())


    All products | Books to Scrape - Sandbox



In [68]:
# extraer solo el texto de head in espacios al inicio o al final
head = soup.find('title')
print(head.get_text(strip=True))

All products | Books to Scrape - Sandbox


Para obtener otro elemento como por ejemplo, el titulo principal, debemos inspeccionar la pagina e identificar el contenedor que lo contiene. 

In [69]:
# extraer titulo principal
titulo = soup.find('div', class_='col-sm-8 h1')
print(titulo)

<div class="col-sm-8 h1"><a href="index.html">Books to Scrape</a><small> We love being scraped!</small>
</div>


In [70]:
titulo = soup.find('div', class_='col-sm-8 h1')
print(titulo.get_text(strip=True))

Books to ScrapeWe love being scraped!


Tambien pudimos haber obtenido el mismo resultado utilizando la clase 'page_inner' debido a que es un contenedor dentro de otro contenedor. Esto se hace por pura coincidencia debido a que el primer div que encuentra es el del titulo. Lo ideal es usar la clase que contiene el texto de interes. 

In [71]:
titulo = soup.find('div', class_='page_inner')
print(titulo.get_text(strip=True))

Books to ScrapeWe love being scraped!


## Extraccion de informacion de productos con HTML y Python

In [72]:
# definimos la url del sitio a scrapear
url = 'https://books.toscrape.com/'

# realizamos la peticion GET
response = requests.get(url)

# parceamos la peticion a formato HTML
soup = BeautifulSoup(response.text, 'html.parser')

In [73]:
# obtener toda la informacion de los productos
products = soup.select('article.product_pod')
print(products)

[<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>, <article class="product_pod">
<div class="image_container">
<a href="catalogue/tipping-the-velvet_999/index.html"><img alt="Tipping the Velvet" class="thumbnail" src="media/cache/2

In [74]:
# extraer solo el nombre, el precio y la url de la imagen

# definimos una lista para almacenar las caracteristicas deseadas
product_list = []

# iteramos sobre cada producto
for product in products:

    # obtenerr nombre
    nombre = product.find('h3').find('a')['title']
    # print(nombre)

    # obtener producto
    precio = product.find('p', class_='price_color').get_text()
    #print(precio) 

    # obtener link imagen
    imagen = product.find('div', class_='image_container').find('img')['src']
    # agregar la raiz a la url para obtener la imagen util
    imagen_url = url + imagen
    #print(imagen_url)

    # guardar informacion en lista
    product_list.append(
        {
            'nombre': nombre,
            'precio': precio,
            'imagen_url': imagen_url
        }
    )
    
print(product_list[1])

{'nombre': 'Tipping the Velvet', 'precio': 'Â£53.74', 'imagen_url': 'https://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg'}


In [75]:
# guardar datos en csv
path_csv = 'resultados/productos.csv'

with open(path_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['nombre', 'precio', 'imagen_url'])
    writer.writeheader()
    writer.writerows(product_list)

print(f'Extraccion completa: {len(product_list)} productos guardados en productos.csv')

Extraccion completa: 20 productos guardados en productos.csv


Vamos agregar a los datos, el numero de estrella del producto y si esta en stock o no 

In [77]:
product_list = []

# iteramos sobre cada producto
for product in products:

    # obtenerr nombre
    nombre = product.find('h3').find('a')['title']
    # print(nombre)

    # obtener producto
    precio = product.find('p', class_='price_color').get_text()
    #print(precio) 

    # obtener link imagen
    imagen = product.find('div', class_='image_container').find('img')['src']
    # agregar la raiz a la url para obtener la imagen util
    imagen_url = url + imagen
    #print(imagen_url)

    # obtener nombre
    estrellas = product.find('p', class_='star-rating').get('class')[1]
    # print(estrellas)

    # obtener stock
    in_stock = product.find('p', class_='instock availability').get_text(strip=True)
    #print(in_stock)
    
    # guardar informacion en lista
    product_list.append(
        {
            'nombre': nombre,
            'precio': precio,
            'imagen_url': imagen_url,
            'calificacion': estrellas,
            'Stock': in_stock
        }
    )

    
print(product_list[1])

{'nombre': 'Tipping the Velvet', 'precio': 'Â£53.74', 'imagen_url': 'https://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg', 'calificacion': 'One', 'Stock': 'In stock'}


In [79]:
# guardar datos en csv nuevamente
path_csv = 'resultados/productos.csv'

with open(path_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['nombre', 'precio', 'imagen_url', 'calificacion', 'Stock'])
    writer.writeheader()
    writer.writerows(product_list)

print(f'Extraccion completa: {len(product_list)} productos guardados en productos.csv')

Extraccion completa: 20 productos guardados en productos.csv


## Scrapear multiples paginas para extraer datos

Realizar scraping multipágina permite recopilar de forma automatizada más información desde un sitio web, especialmente cuando existen límites por página. En el caso de un e-commerce con paginación, esto significa obtener datos de distintos productos de manera eficiente al recorrer múltiples páginas de resultados.

In [80]:
# importar dependencias
import requests
from bs4 import BeautifulSoup

import csv
import json

import time
import random

In [81]:
# se deja el numero de pagina como parametro
base_url = 'https://books.toscrape.com/catalogue/page-{}.html'

In [82]:
# recorrer las primeras 3 paginas y aplicar toda la recoleccion anterior

product_list = []

for page in range(1, 4):
    # obtener cada pagina del sitio web
    url = base_url.format(page)
    # aplicar una peticion GET
    response = requests.get(url)
    # hacer el parseo en formato HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.select('article.product_pod')

    # extraccion de informacion del producto
    for product in products:
        
        nombre = product.find('h3').find('a')['title']
        precio = product.find('p', class_='price_color').get_text()
        imagen = product.find('div', class_='image_container').find('img')['src']
        # agregar la raiz a la url para obtener la imagen util
        imagen_url = url + imagen
        estrellas = product.find('p', class_='star-rating').get('class')[1]
        in_stock = product.find('p', class_='instock availability').get_text(strip=True)

        # guardar informacion en lista
        product_list.append(
            {   
                'nombre': nombre,
                'precio': precio,
                'imagen_url': imagen_url,
                'calificacion': estrellas,
                'Stock': in_stock
            }
        )

    # Espera breve entre paginas para simular navegacion real
    time.sleep(1)
    print(f'Pagina {page} procesada')

Pagina 1 procesada
Pagina 2 procesada
Pagina 3 procesada


In [84]:
# guardar informacion en un nuevo csv
path_csv = 'resultados/productos_multipaginas.csv'

with open(path_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['nombre', 'precio', 'imagen_url', 'calificacion', 'Stock'])
    writer.writeheader()
    writer.writerows(product_list)

print(f'Scraping multipagina completado: {len(product_list)} productos guardados en productos_multipaginas.csv')

Scraping multipagina completado: 60 productos guardados en productos_multipaginas.csv


## Manejo de errores en web scraping con try-except

Implementar técnicas como el uso de estructuras try-except nos permite manejar situaciones específicas, como páginas que no existen (por ejemplo, error 404) o productos incompletos, sin interrumpir el flujo del programa.

Para evitar problemas cuando la paginación excede el número de páginas disponibles, puedes incluir un bloque try-except alrededor de la solicitud GET

In [85]:
# vamos a implementar el scraping en paginas que no existen para manejar los errores
# recorres de la pagina 47 a la 53

product_list = []

for page in range(47, 54):
    # obtener cada pagina del sitio web
    url = base_url.format(page)
    
    try:
        # aplicar una peticion GET
        response = requests.get(url)
        # lanzar error para codigo 400 o 500
        response.raise_for_status()
        # hacer el parseo en formato HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        products = soup.select('article.product_pod')
    except requests.RequestException as e:
        print(f'Error en la pagina {page}: {e}')
        # sigue con la iteracion
        continue

    # manejo de errores al extraer informacion de los productos
    for product in products:
        
        try:
            nombre = product.find('h3').find('a')['title']
            precio = product.find('p', class_='price_color').get_text()
            imagen = product.find('div', class_='image_container').find('img')['src']
            # agregar la raiz a la url para obtener la imagen util
            imagen_url = url + imagen
            estrellas = product.find('p', class_='star-rating').get('class')[1]
            in_stock = product.find('p', class_='instock availability').get_text(strip=True)

            # guardar informacion en lista
            product_list.append(
                {   
                    'nombre': nombre,
                    'precio': precio,
                    'imagen_url': imagen_url,
                    'calificacion': estrellas,
                    'Stock': in_stock
                }
            )
        except Exception as ex:
            print('Error al extraer datos de un producto: ', ex)    

    # Espera breve entre paginas para simular navegacion real
    time.sleep(1)
    print(f'Pagina {page} procesada')

Pagina 47 procesada
Pagina 48 procesada
Pagina 49 procesada
Pagina 50 procesada
Error en la pagina 51: 404 Client Error: Not Found for url: https://books.toscrape.com/catalogue/page-51.html
Error en la pagina 52: 404 Client Error: Not Found for url: https://books.toscrape.com/catalogue/page-52.html
Error en la pagina 53: 404 Client Error: Not Found for url: https://books.toscrape.com/catalogue/page-53.html


In [86]:
# guardar informacion con errores en un nuevo csv
path_csv = 'resultados/productos_con_errores.csv'

with open(path_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['nombre', 'precio', 'imagen_url', 'calificacion', 'Stock'])
    writer.writeheader()
    writer.writerows(product_list)

print(f'Scraping multipagina completado: {len(product_list)} productos guardados en productos_con_errores.csv')

Scraping multipagina completado: 80 productos guardados en productos_con_errores.csv
